# Welcome to the I2K 2022 PyImageJ workshop!

This notebook will guide you through ways to use PyImageJ to connect tools from ImageJ and Fiji with Python-based image processing and data science.

It is structured as a series of exercises, which you will solve on your own. The [accompanying video presentation](https://www.youtube.com/watch?v=dQw4w9WgXcQ) will walk you through this notebook, providing solutions to each exercise, so that you can proceed even if you get stuck at any particular point.

In addition to this workshop, there is also a [sequence of tutorial notebooks](https://github.com/imagej/pyimagej/tree/master/doc#tutorial-notebooks) in the [PyImageJ documentation](https://github.com/imagej/pyimagej#readme).

# ImageJ, ImageJ2, and Fiji: What's the Difference?

Before we begin, it is important to clarify these three terms:

|   |   |
|---|---|
| ![](https://scijava.org/icons/imagej-icon-64.png) | [ImageJ](https://imagej.net/software/imagej) is public domain software for processing and analyzing scientific images, written by Wayne Rasband at the National Institutes of Health. It is a cross-platform desktop application written in Java. |
| ![](https://scijava.org/icons/imagej2-icon-64.png)| [ImageJ2](https://imagej.net/software/imagej2) is a redesign of ImageJ for multidimensional image data, with a focus on scientific imaging. Its central goal is to broaden the paradigm of ImageJ beyond the limitations of the original ImageJ application, to support the next generation of multidimensional scientific imaging. |
| ![](https://scijava.org/icons/fiji-icon-64.png) | [Fiji](https://imagej.net/software/fiji) is a “batteries-included” distribution of ImageJ2, bundling a lot of plugins which facilitate scientific image analysis. |

You can use PyImageJ to interface with both [ImageJ](https://imagej.net/software/imagej) and [ImageJ2](https://imagej.net/software/imagej2) functionality, including [Fiji](https://imagej.net/software/fiji) plugins, as well as plugins from other [ImageJ update sites](https://imagej.net/update-sites/).

## 1. Initializing the ImageJ2 gateway

The first step to using ImageJ technologies from Python is to initialize an ***ImageJ2 gateway*** using the `imagej.init` function.

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 In the following cell, please add code to initialize an ImageJ2 gateway ***with Fiji v2.5.0***, as a new variable called `ij`. Check the [PyImageJ Initialization guide](https://github.com/imagej/pyimagej/blob/1.2.0/doc/Initialization.md) for details. Then execute the cell by pressing <kbd>shift</kbd>+<kbd>enter</kbd>, or by clicking the `▶ Run` button in the toolbar above.

</div>

In [ ]:
# -- EXERCISE 1: Initializing the ImageJ2 gateway --
# Import imagej and call the imagej.init function.
# HINT 1: See "Ways to initialize" in the PyImageJ Initialization guide linked above.
# HINT 2: The first time initializing takes longer since Java libraries are downloaded.


<table><tr><td>

💡 Unfortunately, as of this writing, it is normal to see a bunch of messages like:

> 16:10:18.862 [SciJava-7668d560-Thread-0] DEBUG loci.formats.ClassList - Could not find loci.formats.in.URLReader

These are non-fatal warnings, which you can ignore.

</td></tr></table>

To verify that our gateway is working, we will now perform some sanity checks. Click into the next cell and execute it.

In [ ]:
assert 'ij' in locals(), "The ij variable doesn't exist. Did you assign the created gateway to a variable called ij?"
assert hasattr(ij, 'app'), "The ij variable exists, but does not have the expected structure. Are you sure it is an ImageJ2 gateway object?"
app_versions = {"ImageJ1": "1.53r", "ImageJ2": "2.5.0/1.53r", "Fiji": "2.5.0"}
all_good = True
for name, expected_version in app_versions.items():
    app = ij.app().getApp(name)
    actual_version = app.getVersion() if app else "None"
    assert actual_version == expected_version, f"Expected {expected_version} for the gateway's {name} component, but it was {actual_version}."
print("Your gateway looks good! Nice work!")

If the gateway looks good, carry on. Otherwise, because Java can only be initialized once per Python session, you will need to restart your kernel by choosing "Restart & Clear Output" from the "Kernel" menu, and try again.

## 2. Importing Java classes into Python

PyImageJ is built on the [scyjava](https://pypi.org/project/scyjava) library, which is built on [JPype](https://jpype.readthedocs.io/) and [jgo](https://pypi.org/project/jgo). These libraries enable us to import any available Java class into our Python program, and use its full API, just as we could from within a Java program.

To do this, use the `scyjava.jimport` function with a string argument, passing the full name of the Java class you would like to import, assigning the result to a variable.

For example, to import the `java.lang.System` class, you could write:

```python
from scyjava import jimport
System = jimport('java.lang.System')
```

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 In the following cell, please use `scyjava.jimport` to import the `java.lang.Runtime` class, so that we can check how much memory our Java virtual machine has available.

</div>

In [ ]:
# -- EXERCISE 2: Importing Java classes into Python --
# Import Java's Runtime class using jimport.


Now, let's use that runtime object to inspect the memory available to our Java Virtual Machine. Please execute the next cell.

In [ ]:
print(Runtime.getRuntime().maxMemory() // (2**20), " MB available to Java")

You can use `scyjava.jimport` to import any Java class available on Java's runtime classpath.

## 3. Loading data into ImageJ

Now that we have our ImageJ2 gateway, let's feed it some image data!

The two main ways of doing that are:

1. producing image data on the Java side; or
2. wrapping Python image data into Java.

### 3.1. Producing image data on the Java side

The ImageJ2 gateway is an instance of the [`net.imagej.ImageJ` class](https://javadoc.scijava.org/ImageJ2/net/imagej/ImageJ.html), and therefore supports all the usual ImageJ2 functions available in Java.

<table><tr><td>

💡 For a primer on ImageJ2 gateways, see the [Fundamentals of ImageJ2 notebook](https://github.com/imagej/tutorials/blob/2022-04/notebooks/1-Using-ImageJ/1-Fundamentals.ipynb) from the [ImageJ2 tutorial notebooks](https://github.com/imagej/tutorials/tree/2022-04/notebooks).

</td></tr></table>

One such function is `ij.io().open('/path/to/data.tif')`, to read data from files on disk.

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 Please:

1. Download and unpack the [wtembryo](https://samples.scif.io/wtembryo.zip) sample image from the [SCIFIO sample images](https://scif.io/images).
2. In the next cell, open `wtembryo.mov`, assigning the result to a new variable called `embryo`.

</div>

In [ ]:
# -- EXERCISE 3.1: Producing image data on the Java side --
# Read in wtembryo.mov into a variable called embryo.


<table><tr><td>

💡 It is safe to ignore any `INFO` and `DEBUG` messages above.

</td></tr></table>

Let's take a look at the `embryo` image, to understand its structure.

In [ ]:
def dump_info(image):
    """A handy function to print details of an image object."""
    print(f" name: {image.getName() if hasattr(image, 'getName') else 'N/A'}")
    # TODO: use image.getTitle() for the name if that's available.
    print(f" type: {type(image)}")
    print(f"dtype: {image.dtype if hasattr(image, 'dtype') else 'N/A'}")
    print(f"shape: {image.shape}")
    print(f" dims: {image.dims if hasattr(image, 'dims') else 'N/A'}")

if not 'embryo' in locals(): print("I cannot find any variable called embryo. Did you write `embryo = ...` above?")
elif str(type(embryo)) != "<java class 'net.imagej.DefaultDataset'>": print("The embryo variable does not contain an ImageJ2 Dataset. Did you use the `ij.io().open` function?")
else: dump_info(embryo)

Some important points to note here!

1. The embryo's type is [`net.imagej.DefaultDataset`](https://javadoc.scijava.org/ImageJ2/net/imagej/DefaultDataset.html), a Java class. Notably, this class implements the [`net.imagej.Dataset`](https://javadoc.scijava.org/ImageJ2/net/imagej/Dataset.html) interface, the primary image data type of ImageJ2.

2. The embryo's dtype (i.e. the type of its elements, i.e. what kind of sample values it has) is [`UnsignedByteType`](https://javadoc.scijava.org/ImgLib2/net/imglib2/type/numeric/integer/UnsignedByteType.html), the ImgLib2 class for unsigned 8-bit integer data (i.e. uint8) with integer values in the \[0, 255\] range.

3. The image is 4-dimensional, 320 × 240, 3 channels, and 108 time points.

Make sure you know your data, especially which axes are present and in which order.

<table><tr><td>

💡 We won't cover it in this workshop, but there is also `ij.io().save(image, filepath)` to save your image data to disk.

</td></tr></table>

### 3.2. Wrapping Python image data into Java
In many cases, you will have image data in Python already as a NumPy array, or perhaps as an [xarray](https://docs.xarray.dev/), and want to send it into Java for processing with ImageJ routines. This PyImageJ tutorial assumes you are somewhat familiar with NumPy already; if not, please see e.g. [this W3Schools tutorial on NumPy](https://www.w3schools.com/python/numpy/) to learn about NumPy before proceeding.

In the next cell, we will open an image as a NumPy array using the [scikit-image](https://scikit-image.org/) library.

In [ ]:
import skimage
cells = skimage.data.cells3d()
dump_info(cells)

Because this sample image is a NumPy array, but not an xarray, it does not have dimensional axis labels. However, scikit-image has [defined conventions](https://scikit-image.org/docs/dev/user_guide/numpy_images.html#coordinate-conventions) for the order of dimensions as follows:

    (t, pln, row, col, ch)

Where `t` is time, `pln` is plane/Z, `row` is row/Y, `col` is column/X, and `ch` is channel.

Now that we are armed with that knowledge, notice that `cells` actually has a slightly different dimension order, with planar rather than interleaved channels: `(pln, ch, row, col)`. Let's construct an xarray from this image that includes the correct dimensional axis labels:

In [ ]:
import xarray
xcells = xarray.DataArray(cells, dims=('pln', 'ch', 'row', 'col'))
dump_info(xcells)

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em">

🏋 Now to wrap this image as a Java object. We will use PyImageJ's `ij.py.to_java(data)` function, which wraps the image ***without making a copy***. This means that you can modify the original NumPy array values, and the same changes will automatically be reflected on the Java side as well.

Please try calling `ij.py.to_java` with the `cells` object, and assign it to a new variable called `jcells`.

And while we are at it, let's also convert `xcells` into Java, and call it `jxcells`.
    
</div>

In [ ]:
# -- EXERCISE 3.2: Wrapping Python image data into Java --
# Convert cells to jcells, and xcells to jxcells.


Run the next cell to inspect your handiwork:

In [ ]:
assert jcells.shape == (256, 256, 2, 60)
assert jxcells.dims == ('X', 'Y', 'Channel', 'Z')
print("[jcells]")
dump_info(jcells)
print("\n[jxcells]")
dump_info(jxcells)

Let's take a look at what happened here:

* The `cells`, a numpy array, became an `Img<UnsignedShortType>`, an ImgLib2 image type without metadata.
* The `xcells`, an xarray, became a `Dataset`, an ImageJ2 rich image type with metadata including dimensional labels.

In both cases, notice that the dimensions were reordered. PyImageJ makes an effort to respect the scikit-image dimension order convention where applicable, including ***reordering dimensions between ImageJ2 style and scikit-image style*** when translating images between worlds.

In ImageJ-style terminology, the `ZCYX` order becomes `XYCZ`, because ***ImageJ2 indexes dimensions from fastest to slowest moving***, whereas ***NumPy indexes them from slowest to fastest moving***. In both worlds, iterating over this image's samples will step through columns first, then rows, then channels, then focal planes—the only difference is the numerical order of the dimensions.

These differences between conventions can be confusing, so ***pay careful attention to your data's dimensional axis types***.

Note that `ij.py.to_java` supports various data types beyond only images:

| Python type        | Java type                          | Notes |
|--------------------|------------------------------------|-------|
| `numpy.ndarray`    | `net.imglib2.img.Img`              | An `Img` is both a `RandomAccessibleInterval` and an `IterableInterval`. |
| `xarray.DataArray` | `net.imagej.Dataset`               | A `Dataset` is an `Img` with additional metadata including dimensional axis labels. |
| `int`              | `Integer`, `Long`, or `BigInteger` | Destination Java type depends on magnitude of the integer. |
| `float`            | `Float`, `Double`, or `BigDecimal` | Destination Java type depends on magnitude and precision of the value. |
| `str`              | `String`                           |       |
| `bool`             | `boolean`                          |       |
| `dict`             | `LinkedHashMap`                    | Converted recursively (i.e., elements of the dict are converted too). |
| `set`              | `LinkedHashSet`                    | Converted recursively (i.e., elements of the set are converted too). |
| `list`             | `ArrayList`                        | Converted recursively (i.e., elements of the list are converted too). |

But ***only images are wrapped without copying data***.

## 4. Passing image data from Java to Python

To translate a Java image into Python, use:

```python
python_image = ij.py.from_java(java_image)
```

It's the opposite of `ij.py.to_java`:

* Metadata-rich images such as `Dataset` become xarrays.
* Plain metadata-free images (e.g. `Img`, `RandomAccessibleInterval`) become NumPy arrays.
* Generally speaking, Java types in the table above become the corresponding Python types.

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 In the next cell, please convert `jxcells` back to Python, then call our `dump_info` function on it.

</div>

In [ ]:
# -- EXERCISE 4: Passing image data from Java to Python --
# Convert jxcells back to Java, and print its details using dump_info.


Compare this result to the original Python-side `xcells` and Java-side `jxcells`:

In [ ]:
print("[xcells - original Python image]")
dump_info(xcells)
print("\n[jxcells - after conversion to Java]")
dump_info(jxcells)

Notice the difference? The `xcells` dimension order is ZCYX, which flipped to XYCZ when passed to Java. But when passed back to Python, the C dimension was shuffled to the top. The reason is:
    
***PyImageJ makes a best effort to reorganize image dimensions to match scikit-image conventions.*** The dimensions TZYXC are shuffled to the top with that precedence; any other dimensions are left in place behind those. This behavior is intended to make it easier to use converted Python images with Python-side tools like scikit-image and napari (see also [section 6.4](#6.4.-Displaying-images-via-napari)).

So again: ***pay careful attention to your data's dimensional axis types***.

## 5. Slicing image data

NumPy supports a powerful slicing operation, which lets you focus on a subset of your image data. You can crop image planes, work with one channel or time point at a time, limit the range of focal planes, and more.

If you are not already familiar with NumPy array slicing, please peruse the [NumPy slicing tutorial on W3Schools](https://www.w3schools.com/python/numpy/numpy_array_slicing.asp) to get familiar with it.

Here are some quick examples for a 384 × 256 RGB image:

| Syntax                   | Result                              | Shape         |
|--------------------------|-------------------------------------|---------------|
| `image`                  | the image without any slicing       | (256, 384, 3) |
| `image[:,:,0]`           | channel 0 (red)                     | (256, 384)    |
| `image[:,:,-1]`          | last channel (blue)                 | (256, 384)    |
| `image[96:288,64:192,:]` | cropped 192x128 RGB image, centered | (128, 192, 3) |

PyImageJ supports slicing Java images in the same way, using the NumPy syntax. Just be sure to check the `image.shape` and `image.dims` before slicing, to make sure you are slicing the right thing!

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 In the next cell, please create the following new variables:

* `xcells2d` - a 2D slice of the `xcells` image at its centermost plane (index 30), and first channel.
* `xcells3d` - a 3D slice of the `xcells` image at all planes of the second channel.
* `embryo2d` - a 2D slice of the `embryo` image at the final time point of the first channel.
* `embryo3d` - a 3D slice of the `embryo` image at all time points of the first channel.

As a reminder: `xcells` is an xarray (Python), whereas `embryo` is a `Dataset` (Java), but both should be sliceable just the same.

</div>

In [ ]:
# -- EXERCISE 5: Slicing image data --
# Slice xcells into xcells2d and xcells3d, and embryo into embryo2d and embryo3d.
# HINT: Try printing the shape and dims of each image before slicing.


Run the following cell to check your work:

In [ ]:
def check_slice(name, image, shape, expected, evaluate):
    success = image.shape == shape and all(evaluate(image, k) == v for k, v in expected.items())
    print(name, f"looks good: {image.shape}" if success else "isn't right.")

def xarray_value(image, k): return image[k]
def img_value(image, k): return image[k].get()

check_slice("xcells2d", xcells2d, (256, 256), {(0, 0): 4496, (-1, -1): 3682}, xarray_value)
check_slice("xcells3d", xcells3d, (60, 256, 256), {(0, 0, 0): 5311, (-1, -1, -1): 4031}, xarray_value)
check_slice("embryo2d", embryo2d, (320, 240), {(160, 120): 136, (185, 137): 95}, img_value)
check_slice("embryo3d", embryo3d, (320, 240, 108), {(100, 110, 20): 130, (130, 140, 50): 154}, img_value)

Again, you can check out the [NumPy slicing tutorial on W3Schools](https://www.w3schools.com/python/numpy/numpy_array_slicing.asp) for more details on how to use NumPy slicing.

## 6. Displaying images

There are several nice ways to display images, both inside Jupyter Notebook and via external viewers. This section will explore a few options.

### 6.1. Displaying images via ij.py.show

PyImageJ comes with a built-in function, `ij.py.show(image)`, for showing 2D images. As of this writing, it uses [matplotlib.pyplot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html) internally. This function supports both Python and Java image types, but only works for 2D.

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 In the next cell, try displaying the `xcells2d` and `embryo2d` images using `ij.py.show`.

</div>

In [ ]:
# -- EXERCISE 6.1: Displaying images via ij.py.show --
# Display xcells2d and embryo2d using the ij.py.show function.
# EXTRA: Use a custom color map by passing the cmap parameter.


The `ij.py.show` function also supports passing a custom color map e.g. `cmap='hsv'` with the [same syntaxes as matplotlib](https://matplotlib.org/stable/tutorials/colors/colormaps.html). Give it a try above! But if you need more control over the plot beyond that, you will have to use `pyplot` directly.

### 6.2. Displaying images dynamically with ipywidgets

The ipywidgets library lets us add interactive elements such as sliders to our notebook. Let's build a plane viewer for N-dimensional data. 😎

In [ ]:
def plane(image, pos):
    """
    Slices an image plane at the given position.
    :param image: the image to slice
    :param pos: a dictionary from dimensional axis label to element index for that dimension
    """
    # Convert pos dictionary to position indices in dimension order.
    # See https://stackoverflow.com/q/39474396/1207769.
    p = tuple(pos[image.dims[d]] if image.dims[d] in pos else slice(None) for d in range(image.ndim))
    return image[p]

This `plane` function lets us slice an image using its dimensional axis labels:

In [ ]:
ij.py.show(plane(embryo, {'Channel': 1, 'Time': 58}))

Now that we have this ability, we can script some ipywidget sliders around it:

In [ ]:
import ipywidgets

def _axis_index(image, *options):
    axes = tuple(d for d in range(image.ndim) if image.dims[d].lower() in options)
    if len(axes) == 0:
        raise ValueError(f"Image has no {options[0]} axis!")
    return axes[0]

def ndshow(image, cmap=None, x_axis=None, y_axis=None, immediate=False):
    if not hasattr(image, 'dims'):
        # We need dimensional axis labels!
        raise TypeError("Metadata-rich image required")

    # Infer X and/or Y axes as needed.
    if x_axis is None:
        x_axis = _axis_index(image, "x", "col")
    if y_axis is None:
        y_axis = _axis_index(image, "y", "row")

    # Build ipywidgets sliders, one per non-planar dimension.
    widgets = {}
    for d in range(image.ndim):
        if d == x_axis or d == y_axis:
            continue
        label = image.dims[d]
        widgets[label] = ipywidgets.IntSlider(description=label, max=image.shape[d]-1, continuous_update=immediate)

    # Create image plot with interactive sliders.
    def recalc(**kwargs):
        ij.py.show(plane(image, kwargs), cmap=cmap)
    ipywidgets.interact(recalc, **widgets)

Yeah, it's a lot of code. The plan is for this logic to become part of `ij.py.show`, so that you don't have to cut-and-paste it into your notebooks.

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 In the next cell, try calling `ndshow` with `xcells` or `embryo`.

</div>

In [ ]:
# -- EXERCISE 6.2: Displaying images dynamically with ipywidgets --
# Try calling ndshow on embryo and xcells.


One advantage of this approach is that it supports N-dimensional images directly in the notebook. A downside is that ipywidgets like this are not rendered by most static notebook renderers such as [nbviewer](https://nbviewer.org/) and [GitHub](https://github.com/).

### 6.3. Displaying images via itkwidgets

The [itkwidgets package](https://pypi.org/project/itkwidgets/) provides a powerful viewer for 2D and 3D images, point sets, and geometry. Using it is as simple as importing the `view` function from `itkwidgets`, then calling `view(image)`.

***Note: itkwidgets only works with Python images, not Java images.*** Fortunately, you can use `ij.py.from_java` to convert a Java image to Python before displaying it.

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 In the next cell, give itkwidgets a try with `xcells3d` or `embryo3d`.

</div>

In [ ]:
# -- EXERCISE 6.3: Displaying images via itkwidgets --
# Display xcells3d or embryo3d with itkwidgets.view.
# HINT: Convert Java images first using ij.py.from_java.
# EXTRA 1: Try xcells2d and embryo2d as well, if you like.
# EXTRA 2: One way to change Z depth in 3D is scipy.ndimage.zoom.


### 6.4. Displaying images via napari

<table><tr><td>

😿 This section does not currently work on macOS. See [imagej/pyimagej#197](https://github.com/imagej/pyimagej/issues/197) for details. If you are using macOS, please proceed to [section 7](#7.-Calling-SciJava-scripts).

</td></tr></table>

The [napari](https://napari.org/) package provides an external viewer for N-dimensional image data. Using it is as simple as importing the `view_image` function from `napari`, then calling `view_image(image)`.

***Note:*** Like `itkwidgets`, `napari` ***only supports Python images, not Java images***. You will need to convert them from Java to Python using `ij.py.from_java(image)`.

Also be aware that as of this writing, napari does not recognize xarray dimensional axis labels, and it also wants a ***different dimension order from the scikit-image convention***; for `cells`, using `(ch, pln, row, col)` works well. You can use `numpy.moveaxis` to adjust the dimension order.

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 In the next cell, try out napari with `embryo` or `cells`.

</div>

In [ ]:
# -- EXERCISE 6.4: Displaying images via napari --
# Display cells or embryo using napari.
# HINT: Convert Java images first using ij.py.from_java.
# EXTRA: To make napari happy with cells, use numpy.moveaxis to swap Z and C.


The napari viewer will pop up in a separate window. If you don't see anything, check behind your browser window.

<table><tr><td>

💡 See also [napari-imagej](https://github.com/imagej/napari-imagej), a napari plugin for accessing ImageJ functions via an integrated search widget.

</td></tr></table>

### 6.5. Displaying images via ImageJ

Finally, you can use ImageJ as an external viewer from Python, bringing the full power of ImageJ/ImageJ2/Fiji/etc. to bear on your data.

However, you ***must initialize the ImageJ2 gateway in interactive mode*** to access this power. In PyImageJ's default headless mode, the ImageJ graphical user interface (GUI) cannot be used.

<table><tr><td>

😿 On macOS, for technical reasons, interactive mode [is not available](https://github.com/imagej/pyimagej/blob/1.2.0/doc/Troubleshooting.md#non-blocking-interactive-mode-on-macos) ([1](https://github.com/imagej/pyimagej/issues/23), [2](https://github.com/imagej/pyimagej/issues/185)). But [good scientists use Linux](https://www.nature.com/articles/nbt.2740/), for scientific reproducibility among many other reasons: macOS-based programs tend to stop working within 5-10 years because Apple regularly and intentionally breaks the underlying architecture ([1](https://arstechnica.com/gadgets/2018/01/apple-prepares-macos-users-for-discontinuation-of-32-bit-app-support/), [2](https://www.theverge.com/2019/10/12/20908567/apple-macos-catalina-breaking-apps-32-bit-support-how-to-prepare-avoid-update), [3](https://www.notebookcheck.net/The-end-for-Intel-compatible-x86-apps-on-M1-Macs-macOS-11-3-could-drop-Rosetta-in-certain-countries.526072.0.html), [4](https://news.ycombinator.com/item?id=31135972), [5](https://www.extremetech.com/computing/270902-apple-defends-killing-opengl-opencl-as-developers-threaten-revolt)). Apple also [spies on you by default](https://sneak.berlin/20201112/your-computer-isnt-yours/). If you are using a macOS system to work through this notebook, skip to [section 7](#7.-Image-processing).

</td></tr></table>

In the next cell, we will check the mode of our ImageJ2 gateway.

In [ ]:
# In headless mode, the ImageJ user interface cannot be used.
# Run me to make sure we are not in headless mode!
print("Headless" if ij.ui().isHeadless() else "GUI-enabled!")

If the above cell responded `Headless`, and your operating system is Linux or Windows, you can fix it with these three steps:

1. Choose _Kernel &rarr; Restart and Clear Output_ from the Jupyter notebook menu.
2. Edit the [EXERCISE 1](#1.-Initializing-the-ImageJ2-gateway) cell at the top of this notebook to pass `mode='interactive'` to `imagej.init`.
3. Click into the [EXERCISE 6.5](#6.5.-Displaying-images-via-ImageJ) cell below and choose _Cell &rarr; Run All Above_ from the Jupyter notebook menu.

These steps assume you have successfully completed all the exercise cells until this point.

If all goes well, the `ij.ui().isHeadless()` test above will now say `GUI-enabled!`.

----------------

Now that we have the ImageJ GUI available, we can invoke the relevant functions:

* `ij.ui().show(image)` displays an image in an [image window](https://imagej.net/learn#image-windows). It supports ***Java images only***; convert from Python using `ij.py.to_java`.

* `ij.ui().showUI()` displays the [main window](https://imagej.net/learn#the-main-window). This invocation is optional, but you'll need it to access commands via the [menus](https://imagej.net/learn#the-menu-bar) and [search bar](https://imagej.net/learn#the-search-bar).

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 Please show `jxcells` in an ImageJ window using the `ij.ui().show` function.

</div>

In [ ]:
# -- EXERCISE 6.5: Displaying images via ImageJ --
# Show jxcells in an ImageJ window using the ij.ui().show function.
# EXTRA: Use ij.ui().showUI() to show the main window too, and run some ImageJ commands on your data.


You can jump back and forth between the notebook and external viewers such as ImageJ and napari, executing Python code to change the state of the graphical applications, and vice versa. It is a powerful combination for interactively exploring and analyzing data.

<table><tr><td>

💡 If you close ImageJ's main window, it will quit both Java and Python, killing the notebook session. But don't panic: you can re-run your cells quickly via e.g. _Cell &rarr; Run All Above_.

</td></tr></table>

## 7. Calling SciJava scripts

ImageJ2 is built on [SciJava](https://imagej.net/libs/scijava), which provides foundational infrastructure (application container, plugin framework, etc.). One of SciJava's features is [a powerful scripting framework](https://imagej.net/scripting) enabling ImageJ2 to execute scripts in [various scripting languages](https://imagej.net/scripting#supported-languages).

<table><tr><td>

💡 This workshop assumes a basic level of familiarity with SciJava; if you need a primer, see the [Fundamentals of ImageJ2](https://github.com/imagej/tutorials/blob/2022-04/notebooks/1-Using-ImageJ/1-Fundamentals.ipynb) notebook.

</td></tr></table>

PyImageJ includes a convenience function to invoke such scripts from Python as well: `outputs = ij.py.run_script(language, script, args)`, where `args` is a dictionary of input variable names to values, and `outputs` is a dictionary of output variable names to values.

The following cell gives an example of such a script, written in [Groovy](https://imagej.net/scripting/groovy), which uses [ImageJ Ops](https://imagej.net/libs/imagej-ops) to compute numerical statistics about an image, returning the result as a [SciJava Table](https://github.com/imagej/tutorials/blob/2022-04/notebooks/1-Using-ImageJ/4-Tables.ipynb) object.

In [ ]:
compute_stats_script = """
#@ OpService ops
#@ net.imglib2.RandomAccessibleInterval image
#@output stats

statNames = new org.scijava.table.GenericColumn("Statistic")
statValues = new org.scijava.table.DoubleColumn("Value")
addRow = (n, v) -> { statNames.add(n); statValues.add(v.getRealDouble()) }

addRow("geometricMean", ops.stats().geometricMean(image))
addRow("harmonicMean", ops.stats().harmonicMean(image))
addRow("kurtosis", ops.stats().kurtosis(image))
addRow("max", ops.stats().max(image))
addRow("mean", ops.stats().mean(image))
addRow("median", ops.stats().median(image))
addRow("min", ops.stats().min(image))
addRow("moment1AboutMean", ops.stats().moment1AboutMean(image))
addRow("moment2AboutMean", ops.stats().moment2AboutMean(image))
addRow("moment3AboutMean", ops.stats().moment3AboutMean(image))
addRow("moment4AboutMean", ops.stats().moment4AboutMean(image))
addRow("size", ops.stats().size(image))
addRow("skewness", ops.stats().skewness(image))
addRow("stdDev", ops.stats().stdDev(image))
addRow("sum", ops.stats().sum(image))
addRow("sumOfInverses", ops.stats().sumOfInverses(image))
addRow("sumOfLogs", ops.stats().sumOfLogs(image))
addRow("sumOfSquares", ops.stats().sumOfSquares(image))
addRow("variance", ops.stats().variance(image))

stats = new org.scijava.table.DefaultGenericTable()
stats.add(statNames)
stats.add(statValues)
"""

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 Please execute `compute_stats_script` on `embryo2d` using `ij.py.run_script`, saving the result to a variable called `result`.

</div>

In [ ]:
# -- EXERCISE 7: Calling SciJava scripts --
# Execute compute_stats_script on embryo2d using ij.py.run_script, saving the result as outputs.
# HINT 1: The script language is Groovy.
# HINT 2: Wrap the embryo2d into an args dict.
# EXTRA: Fix the bug in PyImageJ or Pandas! ;-)


Let's check out the result!

In [ ]:
stats = result.getOutput("stats")
assert stats is not None, "The script did not produce any stats!"
assert isinstance(stats, jimport('org.scijava.table.Table')), "The returned stats are not a SciJava table!"
print(f"Java type of stats table = {type(stats)}")
pstats = ij.py.from_java(stats)
print(f"Python type of stats table = {type(pstats)}")
pstats

Here we see a couple of things going on:

* PyImageJ converted the SciJava table into a [Pandas DataFrame](https://www.w3schools.com/python/pandas/pandas_dataframes.asp).
* Something went hilariously wrong with printing the names of the statistics. Probably PyImageJ's table converter did something silly? But it might be a bug in Pandas. The fact that Python strings are iterable character sequences probably has something to do with it. Such is the joy of programming!

<table><tr><td>

💡 For more about using ImageJ Ops with PyImageJ, see [section 8](#8.-Using-ImageJ-Ops).

</td></tr></table>

Finally, you might be wondering why we bothered to code this statistics-calculating logic as a Groovy script, when we could have simply written it directly in Python, thanks to the magic of PyImageJ. It's true, we did not need `run_script` here. But there are reasons you might want to use `run_script`:

1. If you have a pre-existing SciJava script, such as one stored in a file, `run_script` enables you to invoke it directly without translating it to Python.
2. When you call `run_script`, the entire script executes on the Java side in one shot, whereas the equivalent Python code would perform many calls across the Python/Java boundary, each of which takes some extra time and might fail due to bugs in PyImageJ/scyjava/JPype/etc. Therefore, depending on the situation, `run_script` might improve performance and/or correctness.
3. You might want to write a block of code in a different language besides Python (how dare you!).

## 8. Using ImageJ Ops

Next we'll look at a quick image processing example with [ImageJ Ops](https://imagej.net/libs/imagej-ops), ImageJ2's curated collection of image processing routines.

<table><tr><td>

💡 If ImageJ Ops is completely new to you, please read the [ImageJ Ops tutorial notebook](https://github.com/imagej/tutorials/blob/2022-04/notebooks/1-Using-ImageJ/2-ImageJ-Ops.ipynb) to get a feel for Ops before proceeding.

</td></tr></table>

In this scenario, we are interested in running the [tubeness](https://imagej.net/plugins/tubeness) filter to perform some edge detection. We start by finding it in the giant bag of available ops:

In [ ]:
[op for op in ij.op().ops() if "tube" in op]

There it is. What parameters does it take?

In [ ]:
ij.op().help("filter.tubeness")

So we need to pass an image (`RandomAccessibleInterval`), a sigma (`double[]`, Javanese for "list of doubles"), and the spatial calibrations (list of `double`). In a vacuum, passing a calibration of 1 for every dimension would be a good starting point. But it turns out that the tubeness op is smart enough to assume 1s anyway if you pass an empty list, so let's do that by default.

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 Please fill in the body of the `tubeness` function below, using `ij.op().run` to execute the `filter.gauss` op on the given image, with the specified sigma and calibrations, returning the result. Use the `ij.py.jargs` function to wrap the op's three arguments into an `Object[]`.

</div>

In [ ]:
# -- EXERCISE 8: Using ImageJ Ops --
# Write a function to compute tubeness of an image using ImageJ Ops.
# HINT: Use ij.py.jargs to wrap your arguments into a Java Object[].
def tubeness(image, sigma, calibration=[]):
    """Compute the tubeness of an image using ImageJ Ops"""
    # START HERE: Fill in your code!
    

And now, we try it! Let's tubeify the `embryo` with a sigma of 10.

In [ ]:
ij.py.show(tubeness(embryo2d, sigma=10))

It would be nice to try out different sigmas more quickly, wouldn't it? Let's add a slider, like we did in [section 6.2](#6.2.-Displaying-images-dynamically-with-ipywidgets):

In [ ]:
def recalc(**kwargs):
    ij.py.show(tubeness(embryo2d, kwargs['sigma']), cmap='magma')
sigma_slider = ipywidgets.IntSlider(description="sigma", min=1, max=20, value=10, continuous_update=True)
ipywidgets.interact(recalc, sigma=sigma_slider)

[Section 10.2](#10.2.-Segmentation-workflow-with-ImageJ2) uses Ops to segment an image. And please check out the [ImageJ Ops tutorial notebook](https://github.com/imagej/tutorials/blob/2022-04/notebooks/1-Using-ImageJ/2-ImageJ-Ops.ipynb) for many more examples of what Ops can do.


## 9. Working with the original ImageJ

So far, we have limited ourselves to the world of ImageJ2. But PyImageJ also includes support for the original ImageJ.

You can tell whether a Java class is part of ImageJ or ImageJ2 by its package prefix:

|   | Project | Prefix | Example class |
|---|---------|--------|---------------|
| ![](https://scijava.org/icons/imagej-icon-64.png) | [ImageJ](https://imagej.net/software/imagej) | `ij` | `ij.ImagePlus` |
| ![](https://scijava.org/icons/imagej2-icon-64.png) | [ImageJ2](https://imagej.net/software/imagej2) | `net.imagej` | `net.imagej.Dataset` |

The [original ImageJ API](https://javadoc.scijava.org/ImageJ1/) is simpler than [that of ImageJ2](https://javadoc.scijava.org/ImageJ2/) in many ways. However, the design of ImageJ imposes some important restrictions.

**Headless mode.** The original ImageJ is a fundamentally GUI-driven application. Some of its features do not work in [headless mode](https://imagej.net/learn/headless), which (as explained in [section 6.5](#6.5.-Displaying-images-via-ImageJ)) is PyImageJ's default mode. In particular, ***you cannot use `RoiManager` or `WindowManager` headless***. PyImageJ makes a best effort to warn you if you attempt to perform such unsupported operations. For further details, see the section ["The original ImageJ API is limited in headless mode"](https://github.com/imagej/pyimagej/blob/1.2.0/doc/Troubleshooting.md#the-original-imagej-api-is-limited-in-headless-mode) of the PyImageJ Troubleshooting guide.

In [ ]:
# In headless mode, this sort of class access will yield a warning.
print(ij.WindowManager.getCurrentImage())

**One ImageJ-enabled gateway at a time.** ImageJ was not designed to run multiple simultaneous instances in the same JVM, whereas ImageJ2 supports multiple gateways at once.

In [ ]:
def legacy_status(gateway):
    print(f" legacy service: {gateway.legacy}")
    print(f"  legacy active? {gateway.legacy and gateway.legacy.isActive()}")
    print(f"ImageJ2 version: {gateway.getVersion()}")

another_ij = imagej.init()
print("[ij - the original gateway]")
legacy_status(ij)
print("\n[another_ij - a second gateway we constructed just now]")
legacy_status(another_ij)

If you need to construct a second gateway with original ImageJ support, shut down the first one first by calling `ij.dispose()`.

### 9.1. Converting images to ImagePlus

The `ij.py.to_imageplus(image)` function converts an image to an `ij.ImagePlus`.

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 In the following cell, please convert `xcells` into an `ImagePlus` named `imp` using the `ij.py.to_imageplus` function.

</div>

In [ ]:
# -- EXERCISE 9.1: Converting images to ImagePlus --
# Convert xcells to an ImagePlus object named imp.


Let's check the structure of this `imp` and get a few stats:

In [ ]:
assert isinstance(imp, imagej._ImagePlus()), "imp is not an ImagePlus"
dump_info(imp)
print(imp.getAllStatistics())

In interactive mode, you can display an `ImagePlus` using `ij.ui().show(imp)` as described in [section 6.5](#6.5.-Displaying-images-via-ImageJ), or by calling `imp.show()`.

***But beware:*** if you then close the image window, ImageJ will *dispose* the `ImagePlus`, leaving it in an unusable state.

Note that PyImageJ does not implement `dtype`, slicing, or element access for `ImagePlus` objects yet. See [imagej/pyimagej#194](https://github.com/imagej/pyimagej/issues/194) for details.

### 9.2. Converting ImagePlus to other image formats

If you have an `ImagePlus`, but would like something else:

* `ij.py.to_dataset(imp)` will convert it to a `net.imagej.Dataset`.
* `ij.py.from_java(imp)` will convert it to an xarray.

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 In the following cell, please convert `imp` into a `Dataset` called `imp2ds`, and also into an xarray called `imp2py`.

</div>

In [ ]:
# -- EXERCISE 9.2: Converting ImagePlus to other image formats --
# Convert imp to a Dataset called imp2ds, and to an xarray called imp2py.


In [ ]:
assert isinstance(imp2ds, imagej._Dataset()) and imp2ds.shape == (256, 256, 2, 60)
assert isinstance(imp2py, xarray.DataArray) and imp2py.shape == (60, 256, 256, 2)
print("[imp2ds]")
dump_info(imp2ds)
print("\n[imp2py]")
dump_info(imp2py)

### 9.3. Keeping image data in sync with `ij.py.sync_image(imp)`

PyImageJ is built on `imglyb` which allows direct wrapping of NumPy array data into special `RandomAccessibleInterval` structures, avoiding the duplication of data. However, when working with an `ImagePlus` in PyImageJ it is important to understand that the original ImageJ has logic that can cause unexpected behavior when modifying the pixel values of the image (whether by direct modification or through a plugin). Let's look at an example of this in action.

We'll start by loading a new image and converting it to an `ImagePlus`:

In [ ]:
import skimage, xarray
skin = xarray.DataArray(skimage.data.skin(), dims=('row', 'col', 'ch'))
skimp = ij.py.to_imageplus(skin)
def print_3x3_corner():
    print(f"skin: {[int(skin[y,x,0]) for x in range(3) for y in range(3)]}")
    print(f"skimp: {[skimp.getPixel(x,y)[0] for x in range(3) for y in range(3)]}")
print_3x3_corner()
ij.py.show(skin)

To modify the values of our image, we can use the `ImageProcessor` from our `ImagePlus` by calling the [`getProcessor()`](https://imagej.nih.gov/ij/developer/api/ij/ij/ImagePlus.html#getProcessor()) method. `ImageProcessor` has a number of methods available for modifying pixel values; in this case let's use the simple [`set(double)`](https://imagej.nih.gov/ij/developer/api/ij/ij/process/ImageProcessor.html#set(double)) method to set all pixels to `17` (a number unlikely to come up otherwise by chance).

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 In the following cell, please use the `ImageProcessor` of `skimp` to set all pixel values to `17`.

</div>

In [ ]:
# -- EXERCISE 9.3a: Keeping image data in sync --
# Set all pixel values of skimp's active plane to the value 17.


In [ ]:
assert skimp.getPixel(0,0)[0] == 17

Now let's compare the pixel values of the `ImagePlus` and the original:

In [ ]:
print_3x3_corner()
ij.py.show(skin)

Is this what you expected? As you can see, the `ImagePlus` is aware of the updated pixel values, but that change hasn't propagated through to the backing NumPy array. For this reason, we created the `ij.py.sync_image` helper method, to sync `ImagePlus` changes back to ImageJ2 and Python.

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 In the following cell, please sync the `skimp` image using the `ij.py.sync_image` function.

</div>

In [ ]:
# -- EXERCISE 9.3b: Keeping image data in sync --
# Sync skimp back to Python using the `ij.py.sync_image` function.


In [ ]:
assert skin[0,0,0] == 17
print_3x3_corner()
ij.py.show(skin)

Success! Red channel is effectively gone because it's 17s all the way down now.

This step is necessary because the original ImageJ makes a copy of image data from ImgLib2 (and therefore Python) on a plane-by-plane basis, and modifications happen directly on that copied data, rather than on the original data.

### 9.4. Invoking ImageJ plugins

The `ij.py.run_plugin` function lets you run ImageJ plugins from Python.

Running ImageJ plugins from script languages is typically done via `ij.IJ.run(imp, plugin, args)`, where `imp` is the `ImagePlus` to make active during plugin execution, `plugin` is the ***label in the menus*** (as recorded by the [Macro Recorder](https://imagej.net/scripting/macro#the-recorder)), and `args` is a ***single space-separated string of arguments***, e.g. `"path=[/data/image.tif] gamma=2 overwrite"`.

To make this more Pythonic, PyImageJ provides a helper function `ij.py.run_plugin(plugin, args={}, imp=None)` where `args` is a `dict` of variables, and `imp` is an optional `ImagePlus`; if no `imp` is given, the `ij.IJ.run(plugin, args)` method without `ImagePlus` argument is used instead. For example: `ij.py.run_plugin(plugin, {'path': '/data/image.tif', 'gamma': 2, 'overwrite': True})`.

Let's try it. First we create a new `ImagePlus`:

In [ ]:
import skimage
mitepy = skimage.data.human_mitosis()
miteimp = ij.py.to_imageplus(mitepy)
ij.py.show(miteimp)

Now let's blur the image using the "Gaussian Blur..." plugin, which modifies the image in-place.

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 In the following cell, please use `ij.py.run_plugin` to run the `Gaussian Blur...` plugin with a `sigma` of `10`.

</div>

In [ ]:
# -- EXERCISE 9.4: Invoking ImageJ plugins --
# Run the "Gaussian Blur..." plugin with a sigma of 10.
# HINT 1: plugin arguments are passed in a dictionary, i.e. {'key': 'value'}
# HINT 2: don't forget to sync the ImagePlus after running the plugin!


Because we sync'd our `ImagePlus`, we can now view the results of the blur on the original `mitepy` data:

In [ ]:
ij.py.show(mitepy)

### 9.5. Running ImageJ macros

The `ij.py.run_macro` function lets you run [ImageJ macros](https://imagej.net/scripting/macro) from Python.

Before you get too excited, there are some things you should know:
* The macro language does not support the complete ImageJ API, only a set of [built-in functions](https://imagej.nih.gov/ij/developer/macro/functions.html), although there are [ways to work around this](https://imagej.net/scripting/macro#overcoming-limitations).
* Macros are executed by a custom interpreter that is buggier and less well tested than other script languages (Groovy, JRuby, Jython, etc.).
* Macros are not intended to run concurrently, meaning you should only run one macro at a time.
* Macros only support three data types: numbers, strings, and simple arrays. Images are passed via numerical IDs.
* Macros rely on the ***active image***—i.e., the image window currently displayed in the foreground—which is tricky to get right while in headless mode.

<table><tr><td>

💡 See the ["Scripting with ImageJ" living workshop](https://imagej.net/events/presentations#living-workshops) for a primer on ImageJ macro programming.

</td></tr></table>

All of that said, macros are popular because they are simpler and easier to learn than other script languages.

Let's suppose your colleague gave you a macro to clear an image's background and then crop down to the central portion of the image...

In [ ]:
background_clear_and_crop = """
setBatchMode(true);

// Compute image background.
original = getImageID();
run("Duplicate...", " ");
run("Median...", "radius=6");
setAutoThreshold("Li dark");
run("Create Selection");

// Clear background of the original image.
selectImage(original);
run("Restore Selection");
setBackgroundColor(0, 0, 0);
run("Clear Outside");

// Crop to center portion of the image.
x = getWidth() / 4
y = getHeight() / 4
makeRectangle(x, y, x*2, y*2);
run("Crop");
rename(getTitle() + "-cropped")
"""

...and now you want to run this macro from Python with PyImageJ.

<table><tr><td>

💡 A great tool for building up macros is the [Macro Recorder](https://imagej.net/scripting/macro#the-recorder). It can also generate SciJava script code by changing the recorded language to JavaScript. Another powerful tool is the [SciJava Script Editor](https://imagej.net/scripting/script-editor), which has an autocomplete feature useful for exploring available functions.

</td></tr></table>

All we have to do is call `ij.py.run_macro` with this `background_crop_and_clear` macro, right?

Not quite... because we need to set the image to be processed as the ***active image*** first. To do that, you must convert the desired image into an `ImagePlus` and then call `show()` on it. You can do this as a succinct one-liner: `ij.py.to_imageplus(image).show()`. Even in headless mode, the `show()` call is a clue to ImageJ to register and activate this image.

***But here there be dragons:*** There is currently a bug where `show()` triggers a `java.awt.HeadlessException` if there is not already at least one registered `ImagePlus`. So we will now employ a hacky workaround to avoid this problem:

In [ ]:
# HACK: Work around ImagePlus#show() failure if no ImagePlus objects are already registered.
if ij.WindowManager.getIDList() is None:
    ij.py.run_macro('newImage("dummy", "8-bit", 1, 1, 1);')

OK, now we are finally ready to give this macro a try on the skimage `coins` sample image.

In [ ]:
import skimage
coins = skimage.data.coins()
ij.py.show(coins)

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 In the following cell, please convert `coins` to an `ImagePlus` titled `coins` and activate it with `show()`.

</div>

In [ ]:
# -- EXERCISE 9.5a: Running ImageJ macros --
# Convert coins to an ImagePlus titled "coins", and make it the active image.
# HINT: Set the image's title using ImagePlus's setTitle function. 


In [ ]:
assert ij.WindowManager.getImage("coins") is not None

<div style="background: #fdb; box-shadow: 4px 4px 4px #d3d3d3; padding: 0.5em; margin-top: 1em">

🏋 Next, in the following cell, please run the `background_clear_and_crop` macro.

</div>

In [ ]:
# -- EXERCISE 9.5b: Running ImageJ macros --
# Run the background_clear_and_crop macro.


We want to compare the before and after images, but this macro does not output anything explicitly. So what do we do?

Fortunately, we can access the cropped result by grabbing it from the `WindowManager` using its `getImage` function:

In [ ]:
for imp_id in ij.WindowManager.getIDList():
    print(ij.WindowManager.getImage(imp_id))

In [ ]:
coins_cropped = ij.WindowManager.getImage("coins-cropped")
dump_info(coins_cropped)

In [ ]:
ij.py.show(coins_cropped)

Wah! Why isn't it cropped? Bugs galore. It works if we duplicate the image first:

In [ ]:
ij.py.show(coins_cropped.duplicate())

As you can see, using ImageJ macros from Python is full of difficulties and edge cases. Proceed with care, and post on the [Image.sc Forum](https://forum.image.sc/) if stuck.

## 10. Example segmentation workflow

In our final section, we will segment the `cells` image using a traditional [ImageJ segmentation workflow](https://imagej.net/imaging/segmentation#flexible-workflow):

1. Preprocess the image
2. Apply an auto threshold
3. Create and manipulate a mask
4. Create and transfer a selection from the mask to the original image
5. Analyze the resulting data

<table><tr><td>

💡 See the ["Segmentation with ImageJ" living workshop](https://imagej.net/events/presentations#living-workshops) for a primer on segmentation in ImageJ.

</td></tr></table>

We will do the same analysis twice: once with ImageJ, and then again with ImageJ2.

There are no formal exercises in this section—play around with the code however you like! Can you improve the quality of the segmentation?

### 10.1. Segmentation workflow with original ImageJ functions

In [ ]:
# Convert the cells image to ImagePlus.
imp = ij.py.to_imageplus(xcells)
dump_info(imp)

In [ ]:
# Slice out an image plane.
c, z, t = 2, 36, 1
Duplicator = jimport('ij.plugin.Duplicator')
imp2d = Duplicator().run(imp, c, c, z, z, t, t)
ij.py.show(imp2d)

In [ ]:
# Preprocess with edge-preserving smoothing.
ij.IJ.run(imp2d, "Kuwahara Filter", "sampling=10") # Look ma, a Fiji plugin!
ij.py.show(imp2d)

In [ ]:
# Threshold to binary mask.
Prefs = jimport('ij.Prefs')
Prefs.blackBackground = True
ij.IJ.setAutoThreshold(imp2d, "Otsu dark")
mask = imagej._ImagePlus()("mask", imp2d.createThresholdMask())
ij.IJ.run(imp2d, "Close", "")
ij.py.show(mask)

In [ ]:
# Clean up the binary mask.
ij.IJ.run(mask, "Dilate", "")
ij.IJ.run(mask, "Fill Holes", "")
ij.IJ.run(mask, "Watershed", "")
ij.py.show(mask)

In [ ]:
# Transfer the mask back to the original image as a selection.
ij.IJ.run(mask, "Create Selection", "")
imp.setRoi(mask.getRoi()) # analogous to "Restore Selection"
ij.IJ.run(mask, "Close", "")

# Split the ROI into cells.
# This works because cells are disconnected due to the watershed.
roi = imp.getRoi()
roi.setPosition(c, z, t) # TODO: Do we also need imp.setSlice(c, z, t)?
rois = roi.getRois()

# Measure each cell.
ij.IJ.run("Set Measurements...", "area mean min centroid median skewness kurtosis redirect=None decimal=3");
for cell in rois:
    imp.setRoi(cell)
    ij.IJ.run(imp, "Measure", "") # adds a row to the results table

# Convert the measurements to a Pandas DataFrame via a SciJava Table.
# Would be nice to simplify this -- https://github.com/imagej/pyimagej/issues/196
results = ij.ResultsTable.getResultsTable()
table = ij.convert().convert(results, jimport('org.scijava.table.Table'))
measurements = ij.py.from_java(table)
measurements #TODO: table only renders with one row -- clashing row keys?

Notice that we avoided using the `RoiManager`, so that the workflow still works in headless mode.

### 10.2. Segmentation workflow with ImageJ2

In [ ]:
# Slice out an image plane.
c, z = 1, 35
cells_slice = xcells[z,c,:,:]
ij.py.show(cells_slice)
jslice = ij.py.to_java(cells_slice)

In [ ]:
# Preprocess with edge-preserving smoothing.
HyperSphereShape = jimport("net.imglib2.algorithm.neighborhood.HyperSphereShape")
smoothed = ij.op().run("create.img", jslice)
ij.op().run("filter.median", ij.py.jargs(smoothed, cells_slice, HyperSphereShape(5)))
ij.py.show(smoothed)

In [ ]:
# Threshold to binary mask.
mask = ij.op().run("threshold.otsu", smoothed)
ij.py.show(mask)

In [ ]:
# Clean up the binary mask.
mask = ij.op().run("morphology.dilate", mask, HyperSphereShape(1))
mask = ij.op().run("morphology.fillHoles", mask)
ij.py.show(mask)

In [ ]:
# Watershed: mask to labeling.
use_eight_connectivity = True
draw_watersheds = False
sigma = 10
args = ij.py.jargs(None, mask, use_eight_connectivity, draw_watersheds, sigma, mask)
labeling = ij.op().run("image.watershed", args)
ij.py.show(labeling.getIndexImg(), cmap='tab10')

In [ ]:
# Calculate statistics.

from collections import defaultdict
from pandas import DataFrame

Regions = jimport("net.imglib2.roi.Regions")
LabelRegions = jimport("net.imglib2.roi.labeling.LabelRegions")

def compute_stats(region, img, stats):
    samples = Regions.sample(region, img)
    stats["area"].append(ij.op().run("stats.size", samples).getRealDouble())
    stats["mean"].append(ij.op().run("stats.mean", samples).getRealDouble())
    min_max = ij.op().run("stats.minMax", samples)
    stats["min"].append(min_max.getA().getRealDouble())
    stats["max"].append(min_max.getB().getRealDouble())
    centroid = ij.op().run("geom.centroid", region)
    stats["centroid"].append(tuple(centroid.getDoublePosition(d) for d in range(centroid.numDimensions())))
    stats["median"].append(ij.op().run("stats.median", samples).getRealDouble())
    stats["skewness"].append(ij.op().run("stats.skewness", samples).getRealDouble())
    stats["kurtosis"].append(ij.op().run("stats.kurtosis", samples).getRealDouble())

regions = LabelRegions(labeling)
stats = defaultdict(list)
for region in regions:
    compute_stats(region, jslice, stats)
DataFrame(stats)